In [1]:
import pandas as pd
import numpy as np
import folium
from branca.colormap import LinearColormap

In [2]:
df = pd.read_csv('FakeIndividualConstituents.csv')

In [3]:
df

,Postcode,NumberDonations,TotalDonated,AverageDonated,Newsletter
0,NG9 3WF,4,61,15.25,0
1,NG9 4WP,1,21,21.00,0
2,NG9 3EL,1,27,27.00,0
3,NG1 9FH,5,75,15.00,0
4,NG5 6QZ,1,15,15.00,1
...,...,...,...,...,...
95,NG2 1WY,1,15,15.00,0
96,NG8 1ND,10,164,16.40,1
97,NG9 2QA,1,15,15.00,1
98,NG3 1FF,22,333,15.14,1


In [4]:
post = pd.read_csv(r'C:\Users\Daniel\Downloads\open_postcode_geo.csv\open_postcode_geo.csv', header=None)
post = post[[0, 7, 8]]
post = post.rename({0: 'Postcode', 7: 'Latitude', 8: 'Longitude'}, axis=1)

df = df.merge(post, on='Postcode', how='left')

df.head()

,Postcode,NumberDonations,TotalDonated,AverageDonated,Newsletter,Latitude,Longitude
0,NG9 3WF,4,61,15.25,0,52.930121,-1.198353
1,NG9 4WP,1,21,21.00,0,52.921587,-1.247504
2,NG9 3EL,1,27,27.00,0,52.938985,-1.239510
3,NG1 9FH,5,75,15.00,0,52.955008,-1.141045
4,NG5 6QZ,1,15,15.00,1,52.996670,-1.106307


In [5]:
m = folium.Map(location=[52.9548, -1.1581], zoom_start=12)

colors = ['green', 'yellow', 'orange', 'red', 'purple']
linear_colormap = LinearColormap(colors=colors,
                                 index=[0, 100, 250, 500, 1000],
                                 vmin=df['TotalDonated'].min(),
                                 vmax=df['TotalDonated'].quantile(0.991))

for index, row in df.iterrows():    
    num_don = row['NumberDonations']
    total_don = row['TotalDonated']
    news = bool(row['Newsletter'])
    avg_don = row['AverageDonated']
    
    popup_text = f'<div style="width: 175px;">\
                  Total Donated: £{total_don:.2f}<br>\
                  Number of Donations: {num_don}<br>\
                  Average Donation: £{avg_don:.2f}<br>\
                  Subscribed to Newsletter: {news}\
                  </div>'
    
    color = linear_colormap(total_don)
    
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=color, 
        fill=True,
        fill_color=color,
        fill_opacity=1,
        popup=popup_text
    ).add_to(m)

linear_colormap.add_to(m)

m.save('FakeDonorMap_noLayerControl.html')
m

In [6]:
m = folium.Map(location=[52.9548, -1.1581], zoom_start=12)

colors = ['green', 'yellow', 'orange', 'red', 'purple']
linear_colormap = LinearColormap(colors=colors,
                                 index=[0, 100, 250, 500, 1000],
                                 vmin=df['TotalDonated'].min(),
                                 vmax=df['TotalDonated'].quantile(0.991))

# Create FeatureGroups
fgroups = [folium.map.FeatureGroup(name=f"Total Donated:  £{lower}-£{upper}") for lower, upper in zip([0, 100, 250, 500, 750, 1000], [100, 250, 500, 750, 1000, float('inf')])]

for index, row in df.iterrows():    
    num_don = row['NumberDonations']
    total_don = row['TotalDonated']
    news = bool(row['Newsletter'])
    avg_don = row['AverageDonated']
    
    popup_text = f'<div style="width: 175px;">\
                  Total Donated: £{total_don:.2f}<br>\
                  Number of Donations: {num_don}<br>\
                  Average Donation: £{avg_don:.2f}<br>\
                  Subscribed to Newsletter: {news}\
                  </div>'
    
    color = linear_colormap(total_don)
    
    marker = folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=color, 
        fill=True,
        fill_color=color,
        fill_opacity=1,
        popup=popup_text
    )
    
    # Add the marker to the appropriate FeatureGroup
    for fgroup, (lower, upper) in zip(fgroups, zip([0, 100, 250, 500, 750, 1000], [100, 250, 500, 750, 1000, float('inf')])):
        if lower <= total_don < upper:
            fgroup.add_child(marker)
            break

# Add the FeatureGroups to the map
for fgroup in fgroups:
    m.add_child(fgroup)

linear_colormap.add_to(m)
m.add_child(folium.LayerControl())


m.save('FakeDonorMap_withLayerControl.html')
m